# Install Iibrary

In [ ]:
!pip install mysql-connector-python
!pip install pandas
!pip install matplotlib
!pip install SQLAlchemy
!pip install config

## Import Library

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import getpass
from sqlalchemy import create_engine
import datetime
import warnings 
warnings.filterwarnings("ignore")
import mysql.connector
from getpass import getpass

## Define Function

In [ ]:
## Menu Insight 1 (Hotel Traffic)
def insight_1():

# Comparison of weekday traffic in each year
    sql_command1 = """SELECT
                    DISTINCT DAYNAME(full_date) AS weekday,
                    COUNT(CASE WHEN year = '2017' THEN booking_id END) AS '2017',
                    COUNT(CASE WHEN year = '2018' THEN booking_id END) AS '2018',
                    COUNT(booking_id) AS total
                FROM booking_data\
                GROUP BY weekday\
                ORDER BY total DESC;"""

    df = pd.read_sql(sql_command1, my_db)

    df = df.set_index('weekday')
    df.plot.bar(title = "Comparison of weekday traffic in each year", figsize = (15,7), rot = 0)
    plt.show()

# Popular room type in each weekday
    sql_command2 = """WITH t_1 AS (
                    SELECT
                        bdetails.room_type AS room_type,
                        COUNT(CASE WHEN DAYNAME(bdata.full_date) = 'Monday' THEN DAYNAME(bdata.full_date) END) AS Monday,
                        COUNT(CASE WHEN DAYNAME(bdata.full_date) = 'Tuesday' THEN DAYNAME(bdata.full_date) END) AS Tuesday,
                        COUNT(CASE WHEN DAYNAME(bdata.full_date) = 'Wednesday' THEN DAYNAME(bdata.full_date) END) AS Wednesday,
                        COUNT(CASE WHEN DAYNAME(bdata.full_date) = 'Thursday' THEN DAYNAME(bdata.full_date) END) AS Thursday,
                        COUNT(CASE WHEN DAYNAME(bdata.full_date) = 'Friday' THEN DAYNAME(bdata.full_date) END) AS Friday,
                        COUNT(CASE WHEN DAYNAME(bdata.full_date) = 'Saturday' THEN DAYNAME(bdata.full_date) END) AS Saturday, 
                        COUNT(CASE WHEN DAYNAME(bdata.full_date) = 'Sunday' THEN DAYNAME(bdata.full_date) END) AS Sunday,
                        COUNT(bdata.full_date) AS total
                    FROM booking_data AS bdata
                    JOIN booking_details AS bdetails
                    ON bdata.booking_id = bdetails.booking_id
                    GROUP BY room_type
                    ORDER BY room_type)
                SELECT room_type, 
                    Monday + Tuesday + Wednesday + Thursday + Friday as weekday, 
                    Saturday + Sunday as weekend FROM t_1;"""

    df = pd.read_sql(sql_command2, my_db)

    df.plot(title = "Popular room type in each weekday", figsize= (15, 7), kind='bar', x = 'room_type', rot = 0)
    plt.show()
    return
    
## ********************************************************************************************************************* ##

## Menu Insight 2 (The relationship between room type and cancelation status)
def insight_2():

# Lead time and Cancelation Status
    sql_command3 = """SELECT
                        lead_time,
                        COUNT(*) AS total_cancel
                    FROM customer_data
                    WHERE lead_time IS NOT NULL
                    AND status = 'Canceled'
                    GROUP BY lead_time
                    ORDER BY lead_time DESC;"""

    df = pd.read_sql(sql_command3, my_db)

    df.plot(title = "The relationship between lead time and cancelation status", figsize = (15, 7), kind = 'line', x = 'lead_time')
    plt.show()
    
# Market segment and Cancelation Status
    sql_command4 = '''SELECT
                        market_segment,
                        COUNT(status = 'Canceled') AS 'Canceled'
                    FROM customer_data\
                    GROUP BY market_segment\
                    ORDER BY Canceled DESC;'''

    df = pd.read_sql(sql_command4, my_db)

    df.plot(title = "Market segment and Cancelation Status", figsize= (10, 7), kind='bar', x = 'market_segment', rot = 0)
    plt.show()
    return

## ********************************************************************************************************************* ##

## Menu Insight 3 (Hotel Revenue)
def insight_3():
   
# Revenue from each type of room     
    sql_command5 = '''SELECT
                    room_type,
                    ROUND(SUM(avg_euros_per_night), 2) AS total_revenue
                    FROM booking_details AS bdetails\
                    JOIN booking_data AS bdata\
                    ON bdetails.booking_id = bdata.booking_id\
                    GROUP BY bdetails.room_type\
                    ORDER BY ROUND(SUM(avg_euros_per_night), 2) DESC;'''
    df = pd.read_sql(sql_command5, my_db)
    df.plot(title = 'Revenue from each type of room', figsize = (15, 10), kind = 'bar', x = 'room_type',rot= 45, color = 'pink')
    plt.show()
    

# Comparison of revenue in each season    
    sql_command6 = '''SELECT
                    CASE WHEN month BETWEEN 3 AND 5 THEN 'summer'
                    WHEN month BETWEEN 6 AND 10 THEN 'rainy'
                    ELSE 'winter' END AS 'season',
                    ROUND(SUM(avg_euros_per_night), 2) AS 'revenue'
                    FROM booking_data\
                    GROUP BY season\
                    ORDER BY ROUND(SUM(avg_euros_per_night), 2) DESC;'''
    df = pd.read_sql(sql_command6, my_db)
    df = df.set_index('season')
    df.plot(title = 'Comparison of revenue in each season ', figsize = (10, 20), kind= 'pie', y = 'revenue')
    plt.show()
    return

## ********************************************************************************************************************* ##

# Check Availability
def available():
    
    print("Please input date for check availability room: ")
    while True:
        date = input("Date(1-31): ")
        if date.isdigit() and 1 <= int(date) <= 31:
            break
        else:
            print("Invalid input. Please enter a number between 1 and 31")
            
    while True:
        month = input("Month(1-12): ")
        if month.isdigit() and 1 <= int(month) <= 12:
            break
        else:
            print("Invalid input. Please enter a number between 1 and 12")
            
    while True:
        year = input("Year(2017-2025): ")
        if year.isdigit() and 2017 <= int(year) <= 2025:
            break
        else:
            print("Invalid input. Please enter a number between 2017 and 2025")
            
    ymd = f"{year}-{month}-{date}"

    sql_check_available = f'''SELECT
                                    DATE_FORMAT(CONCAT_WS('-', bdata.year, bdata.month, bdata.date),'%Y-%m-%d') AS full_date
                                    , (SELECT room_available FROM room_type WHERE room_type = 'Room_Type 1') - SUM(bdetail.room_type_1) AS r1_available
                                    , (SELECT room_available FROM room_type WHERE room_type = 'Room_Type 2') - SUM(bdetail.room_type_2) AS r2_available
                                    , (SELECT room_available FROM room_type WHERE room_type = 'Room_Type 3') - SUM(bdetail.room_type_3) AS r3_available
                                    , (SELECT room_available FROM room_type WHERE room_type = 'Room_Type 4') - SUM(bdetail.room_type_4) AS r4_available
                                    , (SELECT room_available FROM room_type WHERE room_type = 'Room_Type 5') - SUM(bdetail.room_type_5) AS r5_available
                                    , (SELECT room_available FROM room_type WHERE room_type = 'Room_Type 6') - SUM(bdetail.room_type_6) AS r6_available
                                    , (SELECT room_available FROM room_type WHERE room_type = 'Room_Type 7') - SUM(bdetail.room_type_7) AS r7_available
                                FROM booking_details AS bdetail
                                JOIN booking_data AS bdata
                                ON bdetail.booking_id = bdata.booking_id
                                JOIN room_type AS r
                                ON bdetail.room_type = r.room_type
                                WHERE full_date = "{ymd}"
                                GROUP BY bdata.year, bdata.month, bdata.date;'''

    df = pd.read_sql(sql_check_available, my_db)

    if len(df.values) == 0:
        sql_mock = f'''SELECT
                            "{ymd}" AS full_date
                            , (SELECT room_available FROM room_type WHERE room_type = 'Room_Type 1') AS r1_available
                            , (SELECT room_available FROM room_type WHERE room_type = 'Room_Type 2') AS r2_available
                            , (SELECT room_available FROM room_type WHERE room_type = 'Room_Type 3') AS r3_available
                            , (SELECT room_available FROM room_type WHERE room_type = 'Room_Type 4') AS r4_available
                            , (SELECT room_available FROM room_type WHERE room_type = 'Room_Type 5') AS r5_available
                            , (SELECT room_available FROM room_type WHERE room_type = 'Room_Type 6') AS r6_available
                            , (SELECT room_available FROM room_type WHERE room_type = 'Room_Type 7') AS r7_available'''

        df = pd.read_sql(sql_mock, my_db)
        display(df)
    else:
        sql_check_available = f'''SELECT
                                        DATE_FORMAT(CONCAT_WS('-', bdata.year, bdata.month, bdata.date),'%Y-%m-%d') AS full_date
                                        , (SELECT room_available FROM room_type WHERE room_type = 'Room_Type 1') - SUM(bdetail.room_type_1) AS r1_available
                                        , (SELECT room_available FROM room_type WHERE room_type = 'Room_Type 2') - SUM(bdetail.room_type_2) AS r2_available
                                        , (SELECT room_available FROM room_type WHERE room_type = 'Room_Type 3') - SUM(bdetail.room_type_3) AS r3_available
                                        , (SELECT room_available FROM room_type WHERE room_type = 'Room_Type 4') - SUM(bdetail.room_type_4) AS r4_available
                                        , (SELECT room_available FROM room_type WHERE room_type = 'Room_Type 5') - SUM(bdetail.room_type_5) AS r5_available
                                        , (SELECT room_available FROM room_type WHERE room_type = 'Room_Type 6') - SUM(bdetail.room_type_6) AS r6_available
                                        , (SELECT room_available FROM room_type WHERE room_type = 'Room_Type 7') - SUM(bdetail.room_type_7) AS r7_available
                                    FROM booking_details AS bdetail	
                                    JOIN booking_data AS bdata
                                    ON bdetail.booking_id = bdata.booking_id
                                    JOIN room_type AS r
                                    ON bdetail.room_type = r.room_type
                                    WHERE full_date = "{ymd}"
                                    GROUP BY bdata.year, bdata.month, bdata.date;'''

        df = pd.read_sql(sql_check_available, my_db)
        display(df)
    

    choice=input("Make a reservation ? (Y/N): ")
    while (choice != 'y') and (choice != 'Y') and (choice != 'n') and (choice != 'N'):
        choice=input("Make a reservation ? (Y/N): ")
        
    if (choice=="Y" or choice=="y"):
        insert_table(year,month,date)
        
        print("Record successfully")
        
    else:pass
    
    return
        
## ********************************************************************************************************************* ##

## Insert Table
def insert_table(tmd_year,tmd_month,tmd_day):
    ## Customer Data Table
    # Query customer_data
    sql_get_customer_data = 'SELECT ID FROM dads4002_tmd_sql_hotel.customer_data ORDER BY ID DESC LIMIT 1;'
    ID = ""

    ID = (pd.read_sql(sql_get_customer_data, my_db)).iat[0,0]
    ID = ID[0:3] + str(int(ID[3:]) + 1)

    # Customer_data_insert
    # 1.1. Number of adults
    while True:
        tmd_n_adults = input("Number of adults (ages 18 or above): ")
        if tmd_n_adults.isdigit() and int(tmd_n_adults) >= 0:
            break
        else:
            print("Invalid input. Please enter a integer.")
    
    # 1.2. Number of children
    while True:
        tmd_n_children = input("Number of Children (ages 0-17): ")
        if tmd_n_children.isdigit() and int(tmd_n_children) >= 0:
            break
        else:
            print("Invalid input. Please enter a integer.")
            
    # 1.3. Market Segment
    while True:
        try:
            tmd_market_segment = int(input("Select Market Segment\n1 = Offline\n2 = Online\
            \n3 = Corporate\n4 = Aviation\n5 = Complementary\nEnter number between 1 and 5 : "))
            assert tmd_market_segment in range(1,6)
        except:
            pass
        else:break 
    if tmd_market_segment == 1 : tmd_market_segment = "Offline"
    elif tmd_market_segment == 2 : tmd_market_segment = "Online"
    elif tmd_market_segment == 3 : tmd_market_segment = "Corporate"
    elif tmd_market_segment == 4 : tmd_market_segment = "Aviation"
    else : tmd_market_segment = "Complementary"
            
    ## Booking Data Table
    # Query booking_data
    sql_get_booking_data = 'SELECT booking_id FROM dads4002_tmd_sql_hotel.booking_data ORDER BY booking_id DESC LIMIT 1;'
    booking_id = ""

    booking_id = (pd.read_sql(sql_get_booking_data, my_db)).iat[0,0]
    booking_id = booking_id[0:3] + str(int(booking_id[3:]) + 1)
    
    sql_get_booking_data = 'SELECT customer_id FROM dads4002_tmd_sql_hotel.booking_data ORDER BY customer_id DESC LIMIT 1;'
    customer_id = ""

    customer_id = (pd.read_sql(sql_get_booking_data, my_db)).iat[0,0]
    customer_id = customer_id[0:3] + str(int(customer_id[3:]) + 1)
    
    # Booking_data_insert
    # 2.1. Full Date
    full_date = f"{tmd_year}-{tmd_month}-{tmd_day}"
    
    ## Booking Data Table
    # Query booking_data
    sql_get_booking_data = 'SELECT booking_id FROM dads4002_tmd_sql_hotel.booking_data ORDER BY booking_id DESC LIMIT 1;'
    booking_id = ""

    booking_id = (pd.read_sql(sql_get_booking_data, my_db)).iat[0,0]
    booking_id = booking_id[0:3] + str(int(booking_id[3:]) + 1)

    # Booking_details_insert
    # 3.1. Room Type
    if tmd_market_segment == "Aviation" or tmd_market_segment == "Complementary":
        print("Room type = 3")
        tmd_room_type = "Room_Type 1"
    else :    
        while True:
            tmd_room_type = input("Room type (Plase Select 1-7): ")
            if tmd_room_type.isdigit() and 1 <= int(tmd_room_type) <= 7:
                break
            else:
                print("Invalid input. Please enter a number between 1 and 7.")
                
    if tmd_room_type == "1" : tmd_room_type = "Room_Type 1"
    elif tmd_room_type == "2" : tmd_room_type = "Room_Type 2"
    elif tmd_room_type == "3" : tmd_room_type = "Room_Type 3"
    elif tmd_room_type == "4" : tmd_room_type = "Room_Type 4"
    elif tmd_room_type == "4" : tmd_room_type = "Room_Type 5"
    elif tmd_room_type == "4" : tmd_room_type = "Room_Type 6"
    else : tmd_room_type = "Room_Type 7"
        
    # 3.2. Meal Plan
    while True:
        tmd_meal_plan = input("Meal plan (Please Select 1-3, no plan): ")
        if tmd_meal_plan.isdigit() and (1 <= int(tmd_meal_plan) <= 3) or tmd_meal_plan.lower() == "no plan":
            break
        else:
            print("Invalid input. Please enter a number between 1 and 3, or 'no plan'.")
    
    if tmd_meal_plan == "1" : tmd_meal_plan = "Meal Plan 1"
    elif tmd_meal_plan == "2" : tmd_meal_plan = "Meal Plan 2"
    elif tmd_meal_plan == "3" : tmd_meal_plan = "Meal Plan 3"
    else : tmd_meal_plan = "Not Selected"
    
    # 3.3. Car Parking 
    tmd_car_parking_space = input("Car parking (Please Select Y/N): ")
    while (tmd_car_parking_space != 'y') and (tmd_car_parking_space != 'Y') and (tmd_car_parking_space != 'n') and (tmd_car_parking_space != 'N'):
        tmd_car_parking_space = input("Car parking (Please enter y for Yes or N for No): ")
        
    if (tmd_car_parking_space == "Y" or tmd_car_parking_space == "y"):
        tmd_car_parking_space = "1"
    else:
        tmd_car_parking_space = "0"
        
    # 3.4. Special Request
    tmd_special_requests = input("Special request (Please Select Y/N): ")
    while (tmd_special_requests != 'y') and (tmd_special_requests != 'Y') and (tmd_special_requests != 'n') and (tmd_special_requests != 'N'):
        tmd_special_requests = input("Special request (Please enter y for Yes or N for No): ")
        
    if (tmd_special_requests == "Y" or tmd_special_requests == "y"):
        tmd_special_requests = "1"
    else:
        tmd_special_requests = "0"    
    
    
    # Insert customer data table
    command = (ID, tmd_n_adults,tmd_n_children, tmd_market_segment)
    print(command)
    customer_data = 'INSERT INTO customer_data (ID, n_adults, n_children, market_segment) VALUES ("' + ID + '","' + tmd_n_adults +'" ,"' + tmd_n_children +'" ,"' + tmd_market_segment +'");'

    # Write customer data file
    with open(  f"threemandown_customer_data{datetime.datetime.now().strftime('%Y-%m-%d')}.txt"  ,'w') as file:
        write_string = ','.join([ID, tmd_n_adults, tmd_n_children, tmd_market_segment])
        file.write(write_string)

    # Insert booking data table
    command = (booking_id, customer_id, full_date, tmd_year, tmd_month, tmd_day)
    print(command)
    booking_data = 'INSERT INTO booking_data (booking_id, customer_id, full_date, year, month, date) VALUES ("' + booking_id + '","' + customer_id +'" ,"' + full_date +'" ,"' + tmd_year +'","' + tmd_month +'","' + tmd_day +'" );'

    # Write booking data file
    with open(  f"threemandown_booking_data{datetime.datetime.now().strftime('%Y-%m-%d')}.txt"  ,'w') as file:
        write_string = ','.join([booking_id, customer_id, full_date, str(tmd_year), str(tmd_month), str(tmd_month)])
        file.write(write_string)

    #Insert booking_details table
    command = (booking_id,tmd_meal_plan, tmd_car_parking_space , tmd_room_type, tmd_special_requests)
    print(command)
    booking_details = 'INSERT INTO booking_details (booking_id, meal_plan, car_parking_space , room_type, special_requests) VALUES ("' + booking_id + '","' + tmd_meal_plan +'" ,"' + tmd_car_parking_space +'" ,"' +  tmd_room_type +'","' + tmd_special_requests+'" );'

    # Write booking details file
    with open(  f"threemandown_booking_details{datetime.datetime.now().strftime('%Y-%m-%d')}.txt"  ,'w') as file:
        write_string = ','.join([booking_id, tmd_meal_plan, tmd_car_parking_space, tmd_room_type, tmd_special_requests])
        file.write(write_string)

    my_cursor.execute(customer_data)
    my_cursor.execute(booking_data)
    my_cursor.execute(booking_details)
    my_db.commit()

## ********************************************************************************************************************* ##

## Hotel Function
def threemandown_hotel():
    global my_db
    global my_cursor
    user = input('User name: ')
    pwd = getpass('Password: ')

    my_db = mysql.connector.connect( host = 'localhost',
                                     user = user,
                                     password = pwd,
                                     database = 'dads4002_tmd_sql_hotel',
                                     use_pure = True)
    my_cursor = my_db.cursor()                                 
    print('Successfully connected to the database.')
    print("\n")
    while True:
        print("---------------------------------------")
        print(" Welcome to THREEMANDOWN HOTEL ")
        print("---------------------------------------")
        print("1. Check availability date")
        print("2. INSIGHT 1: Hotel Background")
        print("3. INSIGHT 2: Hotel Revenue")
        print("4. INSIGHT 3: Lead time and Cancelation Status")
        print("5. Exit the program")

        while True:
            try:
                print("\n")
                THREEMANDOWN_HOTEL = int(input("Please select the option (1-5): "))
                assert THREEMANDOWN_HOTEL in range(1,6)

            except:
                print("\n")
                print("Please enter a number between 1 and 5 ")
            else:
                break

        if THREEMANDOWN_HOTEL == 1:
            print("\n")
            available()

        elif THREEMANDOWN_HOTEL == 2:
            print("\n")
            insight_1()

        elif THREEMANDOWN_HOTEL == 3:
            print("\n")
            insight_3()

        elif THREEMANDOWN_HOTEL == 4:
            print("\n")
            insight_2()

        elif THREEMANDOWN_HOTEL == 5:
            my_db.close()
            print('\nDatabase closed.')
            break

        else:pass

    print("\n")       
    print("Exit the program")        
    print("\n")
    print("6520412002 Phummarin Thamrongsath")
    print("6520422029 Jirayut Fukthong")
    print("6520422030 Thanyalak Limsukhawat")
    print("\n")
    
    print("************** THANK YOU **************") 


In [ ]:
threemandown_hotel()